In [1]:
# @everywhere used for the implementation of the modules across multiple workers
# Without it, the code does not run

# Add the modules to the ´read´ path of the Julia
push!(LOAD_PATH, "/scratch/users/vaydingul20/workfolder/COMP541_Project/src/modules/")


# Custom modules
using Network: HapticNet, VisualNet, train_epoch!, save_as_jld2
using TUM69: NetworkData, kfold
using Utils: a_type, notify!
# Ready-to-use modules
using JLD2, Random
using CUDA


┌ Info: Precompiling TUM69 [top-level]
└ @ Base loading.jl:1278


In [19]:
# Set path
path = CUDA.functional() ? "/userfiles/vaydingul20/data/new" : "data/new" # path of the main data
DATA_PATH = isdir(path) && path
# Set pretrained network path
alexnet_PATH = "/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

"/scratch/users/vaydingul20/workfolder/COMP541_Project/alexnet.mat"

In [40]:
nd = NetworkData(DATA_PATH, "accel"; data_type="train", read_type="normal",shuffle=true, read_rate=1.0,  batchsize=500, atype=a_type(Float32))

Train accel data is being loaded!
Test accel data is being loaded!


NetworkData(Tuple{String,Int8}[("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query0.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query1.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query2.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query3.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query4.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query5.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query6.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query7.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterPlate/G1EpoxyRasterPlate_query8.txt", 1), ("/userfiles/vaydingul20/data/new/train/accel/G1EpoxyRasterP

In [21]:
# Seperate into 3 folds for training
kf = kfold(nd; fold=10)
results = []

Any[]

In [22]:
for (ix, (dtrn, dtst)) in enumerate(kf.folds[1:1])
    

    #Reset the model
    hn = HapticNet(;atype=a_type(Float32))

        # Training routine
        res = @time train_epoch!(hn, dtrn, dtst; progress_bar=true, fig=false, info=false, epoch=1)
        # Save model 
        # Add results to the ´results´vector
        push!(results, res)

    
    save_as_jld2(hn, "hn-$ix.jld2")

end

  0.075049 seconds (34.71 k allocations: 1.808 MiB)


In [ ]:
# Save accumulated results
JLD2.@save "results_hn.jld2" results = results

In [41]:
hn = HapticNet(; atype = a_type(Float32))

HapticNet(Network.GCN((Network.Conv(P(KnetArray{Float32,4}(3,3,1,50)), P(KnetArray{Float32,4}(1,1,50,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), (2, 2), true, KnetArray{Float32,N} where N), Network.Conv(P(KnetArray{Float32,4}(3,3,50,100)), P(KnetArray{Float32,4}(1,1,100,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), (2, 2), false, KnetArray{Float32,N} where N), Network.Conv(P(KnetArray{Float32,4}(3,3,100,150)), P(KnetArray{Float32,4}(1,1,150,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), (2, 2), false, KnetArray{Float32,N} where N), Network.Conv(P(KnetArray{Float32,4}(3,3,150,200)), P(KnetArray{Float32,4}(1,1,200,1)), Knet.Ops20.relu, 0, (1, 1), (1, 1), (2, 2), (2, 2), false, KnetArray{Float32,N} where N), Network.Conv(P(KnetArray{Float32,4}(4,12,200,400)), P(KnetArray{Float32,4}(1,1,400,1)), Knet.Ops20.relu, 0, (1, 0), (1, 1), (1, 1), (1, 1), false, KnetArray{Float32,N} where N), Network.Conv(P(KnetArray{Float32,4}(1,1,400,250)), P(KnetArray{Float32,4}(1,1,250,1)), Knet.Ops20

In [42]:
using Knet

In [43]:
@time adam!(hn.model, nd);

Data reading...
(0, 0, true)
(690, 500, false)
4489
(690, 500, false)
(690, 1000, false)
4489
(690, 1000, false)
(690, 1500, false)
4489
(690, 1500, false)
(690, 2000, false)
4489
(690, 2000, false)
(690, 2500, false)
4489
(690, 2500, false)
(690, 3000, false)
4489
(690, 3000, false)
(690, 3500, false)
4489
(690, 3500, false)
(690, 4000, false)
4489
(690, 4000, false)
(690, 4489, true)
4489
 75.926643 seconds (451.17 M allocations: 27.122 GiB, 5.76% gc time)


In [ ]:
hn.model.layers[1].b[:,:,:,:]